<a href="https://colab.research.google.com/github/washmore1/Population-Healthcare-Analysis/blob/main/4_DashboardPrep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Final Analysis and Data Prep for Tableau**

In [1]:
# Setup Installation and Importation
!pip install pandas numpy scikit-learn matplotlib seaborn

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Load the CSV files into Colab
from google.colab import files
uploaded = files.upload()

In [3]:
# Read each csv file into it's own dataframe
df_demo = pd.read_csv('patient_demographics.csv')
df_diag = pd.read_csv('clinical_diagnoses.csv')
df_labs = pd.read_csv('lab_results.csv')
df_meds = pd.read_csv('medications.csv')
df_visits = pd.read_csv('visits.csv')
df_care = pd.read_csv('care_management.csv')
df_risk = pd.read_csv('diabetes_risk.csv')

**Clean and Standardize Tables**

In [4]:
# Ensure PK is a string type for all tables
df_demo['Patient_ID'] = df_demo['Patient_ID'].astype(str)
df_diag['Patient_ID'] = df_diag['Patient_ID'].astype(str)
df_labs['Patient_ID'] = df_labs['Patient_ID'].astype(str)
df_meds['Patient_ID'] = df_meds['Patient_ID'].astype(str)
df_visits['Patient_ID'] = df_visits['Patient_ID'].astype(str)
df_care['Patient_ID'] = df_care['Patient_ID'].astype(str)
df_risk['Patient_ID'] = df_risk['Patient_ID'].astype(str)

In [6]:
# convert all date columns to datetime data type
date_columns = {
    'df_diag': ['Diagnosis_Date'],
    'df_labs': ['Lab_Date'],
    'df_meds': ['Start_Date', 'End_Date'],
    'df_visits': ['Visit_Date'],
    'df_care': ['Start_Date'],
}

dataframes = {
    'df_diag': df_diag,
    'df_labs': df_labs,
    'df_meds': df_meds,
    'df_visits': df_visits,
    'df_care': df_care,
}

for df_name, cols in date_columns.items():
    df = dataframes[df_name]
    for col in cols:
        df[col] = pd.to_datetime(df[col])

**Add Age Bins for Tableau Visuals**

In [7]:
df_demo['Age_Band'] = pd.cut(df_demo['Age'],
                             bins=[18, 30, 45, 60, 75, 90],
                             labels=['18-30', '31-45', '46-60', '61-75', '76-90'],
                             right=True)

**Summarize Lab Results**

In [8]:
# Pivot table for lab means and maxes per patient
df_lab_summary = df_labs.pivot_table(index='Patient_ID',
                                     columns='Lab_Type',
                                     values='Lab_Value',
                                     aggfunc=['mean', 'max']).reset_index()

df_lab_summary.columns = ['Patient_ID',
                          'Mean_Creatinine', 'Mean_HbA1c', 'Mean_LDL Cholesterol',
                          'Max_Creatinine', 'Max_HbA1c', 'Max_LDL Cholesterol']

# Rename columns to simpler names
df_lab_summary.rename(columns={
    'Mean_LDL Cholesterol': 'Mean_LDL',
    'Max_LDL Cholesterol': 'Max_LDL'
}, inplace=True)

**Visit Tendencies Summary**

In [9]:
# Create visit type counts per patient
df_visits_summary = df_visits.groupby(['Patient_ID', 'Visit_Type']).size().unstack(fill_value=0).reset_index()

# Calculate total visits per patient
df_visits_summary['Total_Visits'] = df_visits_summary.loc[:, df_visits_summary.columns != 'Patient_ID'].sum(axis=1)

**Care Management Summary**

In [10]:
# List care types per patient and flag if any care management exists
df_care_summary = df_care.groupby('Patient_ID')['Care_Type'].agg(list).reset_index()
df_care_summary['Has_Care_Management'] = True

**Merge into one, central dataframe for Tableau**

In [12]:
df_dashboard = df_demo.copy()
df_dashboard = df_dashboard.merge(df_risk[['Patient_ID', 'Risk_Score', 'Risk_Category']], on='Patient_ID', how='left')
df_dashboard = df_dashboard.merge(df_lab_summary, on='Patient_ID', how='left')
df_dashboard = df_dashboard.merge(df_visits_summary, on='Patient_ID', how='left')
df_dashboard = df_dashboard.merge(df_care_summary[['Patient_ID', 'Has_Care_Management']], on='Patient_ID', how='left')
df_dashboard['Has_Care_Management'] = df_dashboard['Has_Care_Management'].fillna(False)

<ipython-input-12-d97ebd537712>:6: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_dashboard['Has_Care_Management'] = df_dashboard['Has_Care_Management'].fillna(False)


**Care Gap Analysis**

In [15]:
# Criteria: High risk, no endocrinology visits, no care management
df_dashboard['Care_Gap'] = (
    (df_dashboard['Risk_Category'] == 'High') &
    (df_dashboard.get('Endocrinology', 0) == 0) &
    (~df_dashboard['Has_Care_Management'])
)

**Export Data into CSV's, ready to load in to Tableau**

In [16]:
df_dashboard.to_csv('dashboard_data.csv', index=False)
df_demo.to_csv('patient_demographics.csv', index=False)
df_diag.to_csv('clinical_diagnoses.csv', index=False)
df_labs.to_csv('lab_results.csv', index=False)
df_meds.to_csv('medications.csv', index=False)
df_visits.to_csv('visits.csv', index=False)
df_care.to_csv('care_management.csv', index=False)
df_risk.to_csv('diabetes_risk.csv', index=False)

In [17]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [18]:
!cp dashboard_data.csv "/content/drive/My Drive/PythonHealthcareProject"
!cp patient_demographics.csv "/content/drive/My Drive/PythonHealthcareProject"
!cp clinical_diagnoses.csv "/content/drive/My Drive/PythonHealthcareProject"
!cp lab_results.csv "/content/drive/My Drive/PythonHealthcareProject"
!cp medications.csv "/content/drive/My Drive/PythonHealthcareProject"
!cp visits.csv "/content/drive/My Drive/PythonHealthcareProject"
!cp care_management.csv "/content/drive/My Drive/PythonHealthcareProject"
!cp diabetes_risk.csv "/content/drive/My Drive/PythonHealthcareProject"